In [12]:
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2, boxscoresummaryv2, boxscoreadvancedv2
import requests
import pandas as pd
import sqlite3
import time

In [14]:
# Define the starting season and initialize a list for all box scores
batch_size = 20 # Number of games to process in each batch
delay_between_batches = 10 # Delay (in seconds) between batches
seasons = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']  # Add seasons as needed

# Connect to SQLite database (creates the database if it doesn't exist)
conn = sqlite3.connect("nba_box_scores.db")
cursor = conn.cursor()

#Fetch all games for the specified seasons
all_games = []
for season in seasons:
    print(f"Fetching games for season {season}...")
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable='Regular Season')
    season_games_df = gamefinder.get_data_frames()[0]
    all_games.append(season_games_df)
    time.sleep(10)

# Combine games from all seasons
games_df = pd.concat(all_games, ignore_index=True)
games_df.to_csv('allgames.csv', index=False)  # Save to CSV for reference

# Get unique game IDs
game_ids = games_df['GAME_ID'].unique()

Fetching games for season 2019-20...
Fetching games for season 2020-21...
Fetching games for season 2021-22...
Fetching games for season 2022-23...


In [18]:
# Step 2: Process games in batches
for i in range(0, len(game_ids), batch_size):
    batch = game_ids[i:i + batch_size]
    print(f"Processing batch {i // batch_size + 1} with {len(batch)} games...")

    for game_id in batch:
        try:
            # Fetch box score data for the game
            boxscoretrad = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id, timeout=60)

            # Extract PlayerStats
            player_basic_stats = boxscoretrad.player_stats.get_data_frame()
            player_basic_stats["GAME_ID"] = game_id
            
            # Extract TeamStats
            team_basic_stats = boxscoretrad.team_stats.get_data_frame()
            team_basic_stats["GAME_ID"] = game_id

            # Save data to SQLite database
            player_basic_stats.to_sql("player_basic_stats", conn, if_exists="append", index=False)
            team_basic_stats.to_sql("team_basic_stats", conn, if_exists="append", index=False)


        except Exception as e:
            print(f"Error processing game {game_id}: {e}")
            continue

    # Delay between batches
    print(f"Batch {i // batch_size + 1} completed. Waiting for {delay_between_batches} seconds...")
    time.sleep(delay_between_batches)

# Step 3: Verify Data Saved
print("All data successfully saved to SQLite database!")

Processing batch 1 with 20 games...
Batch 1 completed. Waiting for 10 seconds...
Processing batch 2 with 20 games...
Batch 2 completed. Waiting for 10 seconds...
Processing batch 3 with 20 games...
Batch 3 completed. Waiting for 10 seconds...
Processing batch 4 with 20 games...
Batch 4 completed. Waiting for 10 seconds...
Processing batch 5 with 20 games...
Batch 5 completed. Waiting for 10 seconds...
Processing batch 6 with 20 games...
Batch 6 completed. Waiting for 10 seconds...
Processing batch 7 with 20 games...
Batch 7 completed. Waiting for 10 seconds...
Processing batch 8 with 20 games...
Batch 8 completed. Waiting for 10 seconds...
Processing batch 9 with 20 games...
Batch 9 completed. Waiting for 10 seconds...
Processing batch 10 with 20 games...
Batch 10 completed. Waiting for 10 seconds...
Processing batch 11 with 20 games...
Batch 11 completed. Waiting for 10 seconds...
Processing batch 12 with 20 games...
Batch 12 completed. Waiting for 10 seconds...
Processing batch 13 wi

In [ ]:
# Step 2: Process games in batches
for i in range(0, len(game_ids), batch_size):
    batch = game_ids[i:i + batch_size]
    print(f"Processing batch {i // batch_size + 1} with {len(batch)} games...")

    for game_id in batch:
        try:
            # Fetch box score data for the game
            boxscore = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
            boxscoreadv = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
            boxscoretrad = boxscoretraditionalv2(game_id=game_id)

            # Extract and add relevant data
            game_details = boxscore.game_summary.get_data_frame()
            inactive_players = boxscore.inactive_players.get_data_frame()
            team_stats = boxscoreadv.team_stats.get_data_frame()
            player_stats = boxscoreadv.player_stats.get_data_frame()

            # Tag data with the game ID for clarity
            game_details['GAME_ID'] = game_id
            inactive_players['GAME_ID'] = game_id
            team_stats['GAME_ID'] = game_id
            player_stats['GAME_ID'] = game_id

            # Save data to SQLite database
            game_details.to_sql('game_details', conn, if_exists='append', index=False)
            inactive_players.to_sql('inactive_players', conn, if_exists='append', index=False)
            player_stats.to_sql('player_stats', conn, if_exists='append', index=False)
            team_stats.to_sql('team_stats', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(f"Failed to process game {game_id}: {e}")
            continue

    # Delay between batches to avoid rate limiting
    print(f"Batch {i // batch_size + 1} completed. Waiting for {delay_between_batches} seconds...")
    time.sleep(delay_between_batches)

# Step 3: Verify Data Saved
print("All data successfully saved to SQLite database!")

# Close the database connection
conn.close()